In [1]:
import osmnx as ox
import plotly.graph_objects as go
from shapely.geometry import LineString
import numpy as np
import pickle

In [2]:
# Configure OSMnx to use a valid user agent
ox.settings.log_console = True
ox.settings.use_cache = True  # Cache responses to avoid overloading servers

In [3]:
# Fetch street network (simplify to clean geometries)
place = "Madrid, Spain"
try:
    graph = ox.graph_from_place(place, network_type="drive", simplify=True)
except Exception as e:
    print(f"Failed to fetch graph: {e}")
    exit()

In [4]:
# Get the nodes and edges
with open('./models/optimization/modelo_tabu.pkl', 'rb') as archivo:
    modelo_tabu = pickle.load(archivo)

routes = modelo_tabu["mejor_solucion"] # Lista de coordenadas de la mejor solución

print(routes)

[[(np.float64(40.3885963), np.float64(-3.7270384)), (np.float64(40.3719903), np.float64(-3.6950556)), (np.float64(40.3340239), np.float64(-3.7118486)), (np.float64(40.3885963), np.float64(-3.7270384))], [(np.float64(40.3885963), np.float64(-3.7270384)), (np.float64(40.5124455), np.float64(-3.6778586)), (np.float64(40.4343313), np.float64(-3.7045345)), (np.float64(40.4718405), np.float64(-3.7080787)), (np.float64(40.4678501), np.float64(-3.8145949)), (np.float64(40.3885963), np.float64(-3.7270384))], [(np.float64(40.3885963), np.float64(-3.7270384)), (np.float64(40.4090831), np.float64(-3.676125)), (np.float64(40.4117919), np.float64(-3.6593671)), (np.float64(40.4468717), np.float64(-3.5865153)), (np.float64(40.4347103), np.float64(-3.6422656)), (np.float64(40.4265429), np.float64(-3.6080509)), (np.float64(40.3885963), np.float64(-3.7270384))], [(np.float64(40.3885963), np.float64(-3.7270384)), (np.float64(40.3893878), np.float64(-3.7004158)), (np.float64(40.3862643), np.float64(-3.6908

In [5]:
# Colors and marker symbols
colors = ['blue', 'green', 'red', 'purple', 'orange']
marker_symbols = ['triangle-up', 'circle', 'square']  # Start, Stop, End

In [6]:
# Initialize Plotly figure
fig = go.Figure()

In [9]:
for route_idx, route_points in enumerate(routes):
    route_color = colors[route_idx % len(colors)]
    route_coords = []  # To store all coordinates for the route
    
    # Process each segment (e.g., point1 → point2 → point3)
    for i in range(len(route_points) - 1):
        start_point = route_points[i]
        end_point = route_points[i + 1]
        
        # Find nearest nodes (ensure lon, lat order for OSMnx)
        start_node = ox.nearest_nodes(graph, X=start_point[1], Y=start_point[0])
        end_node = ox.nearest_nodes(graph, X=end_point[1], Y=end_point[0])
        
        # Compute shortest path for this segment
        try:
            segment_path = ox.shortest_path(graph, start_node, end_node, weight="length")
            if not segment_path:
                print(f"No path found for segment {i+1} in Route {route_idx+1}")
                continue
        except ox.exceptions.NoPath:
            print(f"No path for segment {i+1} in Route {route_idx+1}")
            continue
        
        # Extract edges for this segment
        segment_edges = []
        for u, v in zip(segment_path[:-1], segment_path[1:]):
            edge_data = graph.get_edge_data(u, v)[0]
            segment_edges.append((u, v, edge_data))
        
        # Extract coordinates for this segment
        for u, v, edge in segment_edges:
            if 'geometry' in edge:
                line = edge['geometry']
                coords = list(line.coords)
            else:
                # Use node coordinates if geometry is missing
                u_lon, u_lat = graph.nodes[u]['x'], graph.nodes[u]['y']
                v_lon, v_lat = graph.nodes[v]['x'], graph.nodes[v]['y']
                line = LineString([(u_lon, u_lat), (v_lon, v_lat)])
                coords = list(line.coords)
            
            # Convert (lon, lat) → (lat, lon) and clean duplicates
            previous_point = None
            for lon, lat in coords:
                current_point = (lat, lon)
                if current_point != previous_point:
                    route_coords.append(current_point)
                    previous_point = current_point
    
    # Add the entire route as a continuous line
    if route_coords:
        lons = [lon for lat, lon in route_coords]
        lats = [lat for lat, lon in route_coords]
        fig.add_trace(go.Scattermapbox(
            mode="lines",
            lon=lons,
            lat=lats,
            line=dict(width=3, color=route_color),
            name=f"Route {route_idx+1}"
        ))
    
    # Add markers for each stop (start, intermediate, end)
    for point_idx, (lat, lon) in enumerate(route_points):
        fig.add_trace(go.Scattermapbox(
            mode="markers+text",
            lon=[lon],
            lat=[lat],
            marker=dict(size=14, color="black", symbol="circle"),
            text="Pa que sepa",
            textposition="bottom right",
            showlegend=False
        ))


In [11]:
# Configure map layout
all_lats = [point[0] for route in routes for point in route]
all_lons = [point[1] for route in routes for point in route]
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        zoom=11,
        center=dict(lat=np.mean(all_lats), lon=np.mean(all_lons)),
    ),
    margin=dict(l=0, r=0, t=30, b=0),
    height=800,
    width=800,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()